# Batch Normalization

## Introdução

Batch Normalization é uma técnica que permite normalizar automaticamente os valores que atravessam uma camada da rede neural. Normalmente o uso do BatchNorm permite um treinamento mais rápido e com menores cuidados na inicialização dos pesos. Permite que redes profundas sejam treinadas mais facilmente. 

A normalização é feita para que o resultado tenha média zero e variância unitária, porém, em seguida, o resultado é
escalado pelo fator $\gamma$ e somado ao fator $\beta$. Estes dois fatores são parâmetros que serão também otimizados durante o treinamento do gradiente descendente.

Atualmente a técnica de batch normalization é utilizada em todas as redes profundas. 

A camada de batch normalization é colocada entre a camada densa ou convolucional e antes da camada de ativação.

**Referência:**
- Ioffe, S. and Szegedy, C. (2015), Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift., in Francis R. Bach & David M. Blei, ed., 'ICML' , JMLR.org, , pp. 448-456. 
    [PDF:arxiv](http://arxiv.org/pdf/1502.03167.pdf) 

<img src='https://raw.githubusercontent.com/robertoalotufo/files/master/figures/batchnorm_neuronios.png' width="900"></img>

<img src='https://raw.githubusercontent.com/robertoalotufo/files/master/figures/batchnorm_equations.png' width="500"></a>

## Diferença entre fase de treinamento e fase de predição ou avaliação

A normalização ocorre de forma distinta na fase de treinamento e na fase de avaliação:
- em treinamento: a média e variância ($\mu$ e $\sigma^2$) são estimados a partir dos valores
das amostras no mini-batch:
    - `running_mean`
    - `running_var`
- em avaliação: a média e variância é calculada pela média móvel, definida pelo parâmetro momentum ($\lambda$):

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{\mu_t} + (1 - \lambda) \boldsymbol{m}_{t-1} &&
\boldsymbol{v}_{t} =   \lambda \boldsymbol{\sigma^{2}_t} + (1 - \lambda) \boldsymbol{v}_{t-1} 
\end{align*}

## Importações

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display

import sys, copy
import numpy as np
import numpy.random as nr
import pandas as pd

import torch

np.set_printoptions(precision=2, suppress=True)
nr.seed(23456)


## nn.BatchNorm1d - Entrada com dimensão 2

- Documentação oficial: http://pytorch.org/docs/master/nn.html#normalization-layers

Quando a entrada da camada tem duas dimensões (amostras e atributos) a normalização por atributo. Ou seja, calcula-se a estatística (média e variância) para cada coluna da matriz de dados, em cada *mini-batch*.

## Criação rede com apenas uma camada BatchNorm

In [ ]:
torch.manual_seed(1234)

momentum = 0.1
model = torch.nn.BatchNorm1d(5, momentum=momentum)        # Rede formada de apenas uma camada BatchNorm

## Parâmetros iniciais, mv_mean = 0. e mv_var = 1.

In [ ]:
print('Training:', model.training)
gamma, beta, mv_mean, mv_var, _ = copy.deepcopy(model.state_dict()).values()
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: True
weight
[1. 1. 1. 1. 1.]
bias
[0. 0. 0. 0. 0.]
running_mean
[0. 0. 0. 0. 0.]
running_var
[1. 1. 1. 1. 1.]
num_batches_tracked
0


## Entrada x (não normalizada)

In [ ]:
x = torch.randn(4, 5) * 100. # Entrada x
print('x:\n',x)

x:
 tensor([[ -11.1719,  -49.6590,   16.3074,  -88.1688,   28.9097],
        [  48.9871,  -38.5275,  -71.2035,   63.6913,  -71.4080],
        [-108.3124,  -55.4724, -132.4812,   69.6981,  -66.3054],
        [ 121.5757, -252.7335,  147.7784,  -16.9624,  -99.1857]])


## Forward em treinamento: faz predict usando média e var do mini-batch e atualiza mv_mean e mv_var

In [ ]:
print('Training:', model.training)
y = model(x)                           # Forward

Training: True


## Parâmetros: running_mean e running_var estimados com momentum 10% atual e 90% do anterior

In [ ]:
print('Training:', model.training)
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: True
weight
[1. 1. 1. 1. 1.]
bias
[0. 0. 0. 0. 0.]
running_mean
[ 1.28 -9.91 -0.99  0.71 -5.2 ]
running_var
[ 947.05 1054.9  1478.69  560.11  312.71]
num_batches_tracked
1


## Conferindo os novos valores de running_mean e running_var

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{\mu_t} + (1 - \lambda) \boldsymbol{m}_{t-1} &&
\boldsymbol{v}_{t} =   \lambda \boldsymbol{\sigma^{2}_t} + (1 - \lambda) \boldsymbol{v}_{t-1} 
\end{align*}

In [ ]:
print('old mv_mean, mv_var:', mv_mean.numpy(), mv_var.numpy())

mv_mean_new = momentum * x.data.mean(0) + (1 - momentum) * mv_mean
mv_var_new  = momentum * x.data.var(0,unbiased=True) + (1 - momentum) * mv_var

print('my new running mean:',mv_mean_new.numpy())
print('my new running var :',mv_var_new.numpy())

old mv_mean, mv_var: [0. 0. 0. 0. 0.] [1. 1. 1. 1. 1.]
my new running mean: [ 1.28 -9.91 -0.99  0.71 -5.2 ]
my new running var : [ 947.05 1054.9  1478.69  560.11  312.71]


## Saída  y

In [ ]:
y

tensor([[-0.2842,  0.5561,  0.2489, -1.4705,  1.6731],
        [ 0.4299,  0.6813, -0.5823,  0.8744, -0.4014],
        [-1.4374,  0.4907, -1.1644,  0.9671, -0.2959],
        [ 1.2916, -1.7280,  1.4977, -0.3710, -0.9758]],
       grad_fn=<NativeBatchNormBackward>)

## Conferindo com código próprio

Em treinamento, a normalização é feita com a estatística do mini-batch

$$  y = \frac{x - mean[x]}{ \sqrt{Var[x] + \epsilon}} * gamma + beta $$

In [ ]:
mean = x.mean(dim=0).data
var  = x.var(dim=0, unbiased=False).data # Veja que aqui é unbiased
print('mean:',mean.numpy())
print('var: ',var.numpy())
y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)

mean: [ 12.77 -99.1   -9.9    7.06 -52.  ]
var:  [ 7096.13  7905.02 11083.39  4194.05  2338.55]
tensor([[-0.2842,  0.5561,  0.2489, -1.4705,  1.6731],
        [ 0.4299,  0.6813, -0.5823,  0.8744, -0.4014],
        [-1.4374,  0.4907, -1.1644,  0.9671, -0.2959],
        [ 1.2916, -1.7280,  1.4977, -0.3710, -0.9758]])


## Modo predict sem treinamento

In [ ]:
model.training = False
y = model(x)                           # Forward

Observe que agora, o running_mean e running_var serão utilizados

In [ ]:
print('Training:', model.training)
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

Training: False
weight
[1. 1. 1. 1. 1.]
bias
[0. 0. 0. 0. 0.]
running_mean
[ 1.28 -9.91 -0.99  0.71 -5.2 ]
running_var
[ 947.05 1054.9  1478.69  560.11  312.71]
num_batches_tracked
1


## Resultado da rede no modo eval(), note que os valores não ficaram tão normalizados (por quê?)

In [ ]:
y

tensor([[-0.4045, -1.2238,  0.4498, -3.7553,  1.9289],
        [ 1.5503, -0.8811, -1.8259,  2.6613, -3.7441],
        [-3.5611, -1.4028, -3.4195,  2.9151, -3.4555],
        [ 3.9091, -7.4763,  3.8688, -0.7466, -5.3149]],
       grad_fn=<NativeBatchNormBackward>)

## Conferindo com código próprio, note que usa-se o running_mean e running_var

In [ ]:
gamma, beta, mv_mean, mv_var, _ = model.state_dict().values()

mean = mv_mean
var  = mv_var

y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)

tensor([[-0.4045, -1.2238,  0.4498, -3.7553,  1.9289],
        [ 1.5503, -0.8811, -1.8259,  2.6613, -3.7441],
        [-3.5611, -1.4028, -3.4195,  2.9151, -3.4555],
        [ 3.9091, -7.4763,  3.8688, -0.7466, -5.3149]])


## Exercícios

1. Coloque a rede no modo treinamento em um laço, de modo que a running_mean e running_var se aproximem da média e variância do mini-batch. Quantas execuções da rede serão necessárias?

In [ ]:
model.training = True
#for i in range(100):
    #y = model(x)
    # Busque aqui os valores da média móvel da média e variância que estão sendo aprendidos
    #
    #
    #print(mv_mean.numpy(), mv_var.numpy())